In [1]:
from keras.models import Sequential
import tensorflow as tf
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
#from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import random
from PIL import Image
import theano

In [2]:
CATEGORIES = ["DEPRESSED","HEALTHY"]

In [3]:
training = []
def createTrainingData(category):  
  class_num = CATEGORIES.index(category)
  if class_num==0:
    for i in range(101,129):
      j=0
      while True:
        try:
          filename=r"/content/drive/MyDrive/Depression_Current/OpenSmilePlots/F0semitoneFrom27.5Hz_sma3nz/Depressed/AI_"+str(i)+"_result_slice_"+str(j)+"_lowerlevel"+".png"
          img_array = cv2.imread(filename)
          new_array = cv2.resize(img_array, (420,720))
          training.append([new_array, class_num])
          j+=70000
        except:
          break
  elif class_num==1:
    for i in range(301,331):
      j=0
      while True:
        try:
          filename=r"/content/drive/MyDrive/Depression_Current/OpenSmilePlots/F0semitoneFrom27.5Hz_sma3nz/Healthy/AIC_"+str(i)+"_result_slice_"+str(j)+"_lowerlevel"+".png"
          img_array = cv2.imread(filename)
          new_array = cv2.resize(img_array, (420,720))
          training.append([new_array, class_num])
          j+=70000
        except:
          break
  else:
    print("Incorrect input")
createTrainingData("DEPRESSED")

In [4]:
len(training)

419

In [5]:
createTrainingData("HEALTHY")

In [6]:
len(training)

828

In [7]:
random.shuffle(training)
X =[]
y =[]
for features, label in training:
  X.append(features)
  y.append(label)
X = np.array(X).reshape(-1,420,720,3)

X = X.astype('float32')
X /= 255
from keras.utils import np_utils
Y = np_utils.to_categorical(y, 2)
Y=np.array(Y)

print(Y)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 4)

In [9]:
#Exploratory Model
model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(32,(3,3),padding='same',activation=tf.nn.leaky_relu,input_shape=(420,720,3)),
        tf.keras.layers.MaxPooling2D((2, 2), strides=2),
        tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.leaky_relu),
        tf.keras.layers.MaxPooling2D((2, 2), strides=2),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation=tf.nn.leaky_relu),
        tf.keras.layers.Dense(2,  activation=tf.nn.sigmoid)
])

In [10]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [11]:
history=model.fit(X_train, y_train, batch_size =5, epochs =15, verbose = 1, validation_data = (X_test, y_test))

Epoch 1/15
133/133 [==============================] - 368s 3s/step - loss: 3.2384 - accuracy: 0.5710 - val_loss: 0.9659 - val_accuracy: 0.4639
Epoch 2/15
133/133 [==============================] - 397s 3s/step - loss: 0.6173 - accuracy: 0.6586 - val_loss: 0.5314 - val_accuracy: 0.7530
Epoch 3/15
133/133 [==============================] - 402s 3s/step - loss: 0.6194 - accuracy: 0.7175 - val_loss: 0.5521 - val_accuracy: 0.6928
Epoch 4/15
133/133 [==============================] - 384s 3s/step - loss: 0.3272 - accuracy: 0.8958 - val_loss: 0.5584 - val_accuracy: 0.7108
Epoch 5/15
133/133 [==============================] - 384s 3s/step - loss: 0.2076 - accuracy: 0.9517 - val_loss: 0.6995 - val_accuracy: 0.6325
Epoch 6/15
133/133 [==============================] - 397s 3s/step - loss: 0.0965 - accuracy: 0.9909 - val_loss: 0.8436 - val_accuracy: 0.7349
Epoch 7/15
133/133 [==============================] - 385s 3s/step - loss: 0.1189 - accuracy: 0.9743 - val_loss: 0.8307 - val_accuracy: 0.6867

In [12]:
model.save((r"/content/drive/MyDrive/Depression_Current/OpenSmilePlots/F0semitoneFrom27.5Hz_sma3nz/trained_1"))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Depression_Current/OpenSmilePlots/F0semitoneFrom27.5Hz_sma3nz/trained_1/assets


In [13]:
with open(r"/content/drive/MyDrive/Depression_Current/OpenSmilePlots/F0semitoneFrom27.5Hz_sma3nz/Accuracy.txt", "w") as f:
  f.write("The accuracy was 70.48%")